# Map of flights

Creates plot with the flight track on a map together with the sea ice concentration.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import xarray as xr
from lizard.ac3airlib import get_all_flights
from lizard.readers.amsr2_sic_track import read_amsr2_sic_track
from lizard.readers.gps_ins import read_gps_ins
from lizard.readers.mean_sea_ice import read_mean_sea_ice
from lizard.writers.figure_to_file import write_figure

from si_clouds.io.readers.ancillary import read_ancillary_data
from si_clouds.io.readers.retrieval_times import read_retrieval_times

In [ ]:
ds_anc = read_ancillary_data()

In [ ]:
# all-sky times
df_all_sky = read_retrieval_times("all_sky")
time_all_sky = df_all_sky.iloc[:, 0].values

# clear-sky times
df_clear_sky = read_retrieval_times("clear_sky_kt19")
time_clear_sky = df_clear_sky.iloc[:, 0].values

# case times
df_case2 = read_retrieval_times("HALO-AC3_HALO_RF04_hl08")
time_case2 = df_case2.iloc[:, 0].values

df_case1 = read_retrieval_times("HALO-AC3_HALO_RF18_hl03")
time_case1 = df_case1.iloc[:, 0].values

In [ ]:
# get times where the retrieval is valid
time_all_sky = time_all_sky[ds_anc.ix_retrieval_valid.sel(time=time_all_sky).values]
time_clear_sky = time_clear_sky[ds_anc.ix_retrieval_valid.sel(time=time_clear_sky).values]
time_case2 = time_case2[ds_anc.ix_retrieval_valid.sel(time=time_case2).values]
time_case1 = time_case1[ds_anc.ix_retrieval_valid.sel(time=time_case1).values]

In [ ]:
ds_mean_sic = read_mean_sea_ice("HALO-AC3")

flight_ids = get_all_flights("HALO-AC3", "HALO")
lst_gps = []
lst_sic = []
for flight_id in flight_ids:
    ds_gps = read_gps_ins(flight_id)[["lon", "lat"]]
    ds_sic_track = read_amsr2_sic_track(flight_id)
    lst_gps.append(ds_gps)
    lst_sic.append(ds_sic_track)
ds_gps = xr.concat(lst_gps, dim="time")
ds_sic = xr.concat(lst_sic, dim="time")

In [ ]:
proj = ccrs.NorthPolarStereo(central_longitude=0)
data_crs = ccrs.PlateCarree()

z_sic = 0
z_land = 1
z_track = 2
z_grid = 3
z_sondes = 4

fig, ax = plt.subplots(
    1, 1, figsize=(6, 4), subplot_kw={"projection": proj}, layout="constrained"
)
ax.set_extent([-24, 24, 73.5, 90])

land_10m = cfeature.NaturalEarthFeature(
    category="physical",
    name="land",
    scale="10m",
    edgecolor="#7AB9DC",
    linewidth=0.5,
    facecolor="#CAE9FB",
)
ax.add_feature(land_10m, zorder=z_land)

ax.gridlines(
    crs=data_crs,
    draw_labels=["left", "bottom"],
    xlocs=mticker.FixedLocator(np.arange(-180, 180, 10)),
    ylocs=mticker.FixedLocator(np.arange(70, 90, 2)),
    x_inline=False,
    y_inline=False,
    rotate_labels=False,
    linewidth=0.25,
    color="#1F7298",
    alpha=0.5,
    zorder=z_grid,
)

# mean sea ice
im = ax.pcolormesh(
    ds_mean_sic.lon,
    ds_mean_sic.lat,
    ds_mean_sic.sic,
    cmap="Blues_r",
    vmin=0,
    vmax=100,
    transform=data_crs,
    zorder=z_sic,
    shading="nearest",
)
fig.colorbar(
    im,
    ax=ax,
    label="Sea ice concentration [%]",
)

linewidth = 2
s_track = 2

# all over sea ice, potentially no mw data
ax.scatter(
    ds_gps.lon.sel(time=time_all_sky),
    ds_gps.lat.sel(time=time_all_sky),
    color="k",
    lw=0,
    s=s_track,
    transform=data_crs,
    zorder=z_track,
)
ax.scatter(
    ds_gps.lon.sel(time=time_clear_sky),
    ds_gps.lat.sel(time=time_clear_sky),
    color="#bbbbbb",
    lw=0,
    s=s_track,
    transform=data_crs,
    zorder=z_track,
)

# case study
color_case_1 = "green"
color_case_2 = "coral"
ax.scatter(
    ds_gps.lon.sel(time=time_case1),
    ds_gps.lat.sel(time=time_case1),
    color=color_case_1,
    lw=0,
    s=s_track * 2,
    transform=data_crs,
    zorder=z_track,
)

ax.scatter(
    ds_gps.lon.sel(time=time_case2),
    ds_gps.lat.sel(time=time_case2),
    color=color_case_2,
    lw=0,
    s=s_track * 2,
    transform=data_crs,
    zorder=z_track,
)

# make a fake legend with lines
ax.plot([], [], color="k", label="Cloudy", linewidth=s_track)
ax.plot([], [], color="#bbbbbb", label="Clear-sky", linewidth=s_track)
ax.plot([], [], color=color_case_1, label="Case 1", linewidth=s_track * 2)
ax.plot([], [], color=color_case_2, label="Case 2", linewidth=s_track * 2)

ax.legend(
    loc="upper center", frameon=False, ncol=2, bbox_to_anchor=(0.5, -0.06)
)

write_figure(fig, "paper/fig01.png", dpi=600, bbox_inches="tight")
plt.close()